<a href="https://colab.research.google.com/github/Ebenx007/compchem-Compsci-shared-rep/blob/main/2_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from pathlib import Path
import sys
import os
import shutil
import tarfile
import zipfile
import subprocess
import pickle
import re
import glob

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


#   RESTORE RAW DATA FROM [DATA AQUISITION DIRECTORY](https://github.com/Ebenx007/compchem-Compsci-shared-rep/blob/main/1_data_acquisition.ipynb)
---
---

In [3]:
#Load pickled metadata
def load_data(file):
 with open(file,"rb") as f:
   data = pickle.load(f)
 return(data)

In [4]:
!cp "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/raw/raw_data.tar" .

In [5]:
with tarfile.open('raw_data.tar', 'r') as f:
  f.extractall()

In [6]:
#copy raw_data and metadata into pwd 
%cp -r /content/data_acquisition/* .

In [7]:
!ls

data_acquisition			   ProgramData
drive					   programs.tar.gz
juliet_dataset_CWE_ls.pkl		   raw_data.tar
juliet_dataset_CWE_testcases_paths_ls.pkl  sample_data
juliet_dataset_ls.pkl			   submissions_ls.pkl
Juliet_Test_Suite_v1.3_for_C_Cpp.zip	   submissions_tasks_ls.pkl


# 2.  DATA PREPROCESSING 

---


##2.1 Clean Main Dataset:

*   Remove Uncompilable Source Code (platform: colab(linux))
*   Remove Non UTF8 decodable entries



In [8]:
active_dir = '/content/ProgramData/'
test_file_counter = 0
for path, subdirs, files in os.walk(active_dir):
    for name in files:
      test_file_counter +=1
print("{} files extracted from Programming Competition Submissions archive. ".format(test_file_counter))

52000 files extracted from Programming Competition Submissions archive. 


In [9]:
#Variables to hold Meta Data from programming competition dataset preprocessing 
utf8_decodable_submisisons = []
utf8_undecodable_submissions = []
compiled_cpp_code = []
compiled_c_code = []
uncompilable_utf8_decodable_submissions = []

> * Extract contents of .txt files submitted for Programming Competitions, convert to .cpp or .c files, and compile.

In [10]:
submissions_ls = load_data('submissions_ls.pkl')

###2.1.1 Exploration: Clean/Compile 10 samples from Main Dataset:

In [11]:
# 10 Sample to test extraction, conversion and compiling code  
cpp_distinguishing_features = ['.cpp','cin','cout']
for i in range(10):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
    except:
       pass 
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp" 
      # the -4 for the .txt
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      print(new_cpp_filename)

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      print(new_c_filename)



ProgramData/1/397.cpp
ProgramData/1/147.cpp
ProgramData/1/131.cpp
ProgramData/1/1076.c
ProgramData/1/2272.cpp
ProgramData/1/703.cpp
ProgramData/1/2063.c
ProgramData/1/1892.c
ProgramData/1/889.cpp
ProgramData/1/1342.cpp


In [ ]:
#Exploration 
!ls ./ProgramData/1/

In [13]:
#Extract convert and compile the rest of the submissions
cpp_distinguishing_features = ['.cpp','cin','cout']
print('Compiling the rest of the submitted code ...')
for i in range(len(submissions_ls)):
  with open(os.path.join('', submissions_ls[i]), 'r') as f:
    try:
      file_content = f.read()
      utf8_decodable_submisisons.append(submissions_ls[i])
    except:
       utf8_undecodable_submissions.append(submissions_ls[i])
    if any(x in file_content for x in cpp_distinguishing_features):
      new_cpp_filename = submissions_ls[i][:-4] + ".cpp"
      with open(os.path.join('', new_cpp_filename), 'w') as fp:
        cppheader= '''
        #include<iostream> 
        #include <math.h>
        using namespace std;
        '''
        fp.write(cppheader)
        fp.write(file_content)
      subprocess.call(["g++", new_cpp_filename, "-o", submissions_ls[i][:-4] + '.cppout',"-std=c++11", '-w', '-Ofast'])
      compiled_cpp_code.append(new_cpp_filename)
  

    else:
      new_c_filename = submissions_ls[i][:-4] + ".c"
      with open(os.path.join('', new_c_filename), 'w') as fp:
        fp.write(file_content)
      subprocess.call(["gcc", new_c_filename, "-o"+submissions_ls[i][:-4] + '.out',"-std=c99", '-w', '-Ofast'])
      compiled_c_code.append(new_c_filename)
print('...' )
print('Code compilation completed')

Compiling the rest of the submitted code ...
...
Code compilation completed


In [14]:
#Exploration
!ls ./ProgramData/

1    104  16  21  27  32  38  43  49  54  6   65  70  76  81  87  92  98
10   11   17  22  28  33  39  44  5   55  60  66  71  77  82  88  93  99
100  12   18  23  29  34  4   45  50  56  61  67  72  78  83  89  94
101  13   19  24  3   35  40  46  51  57  62  68  73  79  84  9   95
102  14   2   25  30  36  41  47  52  58  63  69  74  8   85  90  96
103  15   20  26  31  37  42  48  53  59  64  7   75  80  86  91  97


In [ ]:
#Exploration
!ls ./ProgramData/104

In [16]:
#Exploration
print("Overview of programming Competition dataset")
print("\n Source code submissions {0}".format(len(submissions_ls)))
print("\n Utf8 Decodable.txt files directories in dataset {0}".format(len(utf8_decodable_submisisons)))
print("\n Utf8 Undecodable .txt files in dataset {0}".format(len(utf8_undecodable_submissions)))
print("\n Compiled .cpp files in dataset {0}".format(len(compiled_cpp_code)))
print("\n Compiled .c files in dataset {0}".format(len(compiled_c_code)))
print("\n Uncompilable utf8 decodable files in dataset {0}".format(len(uncompilable_utf8_decodable_submissions)))

Overview of programming Competition dataset

 Source code submissions 52000

 Utf8 Decodable.txt files directories in dataset 51752

 Utf8 Undecodable .txt files in dataset 248

 Compiled .cpp files in dataset 15406

 Compiled .c files in dataset 36594

 Uncompilable utf8 decodable files in dataset 0


In [17]:
print('Number of Compiled C code files and visual of List')
print(len(compiled_c_code))

Number of Compiled C code files and visual of List
36594


In [ ]:
#Exploration
print(compiled_c_code)

In [19]:
print('Number of Compiled C++ code files and visual of List')
print(len(compiled_cpp_code))

Number of Compiled C++ code files and visual of List
15406


In [ ]:
print(compiled_cpp_code)


> *  Pickle Competition Dataset metadata for later use

In [21]:
with open('cpp_compiled_ls.pkl', 'wb') as fp:
  pickle.dump (compiled_cpp_code, fp)

In [22]:
with open('submissions_ls.pkl', 'wb') as fp:
  pickle.dump (submissions_ls, fp)

In [23]:
with open('decodable_submisisons_ls.pkl', 'wb') as fp:
  pickle.dump (utf8_decodable_submisisons, fp)

In [24]:
with open('c_compiled_ls.pkl', 'wb') as fp:
  pickle.dump (compiled_c_code, fp)

In [25]:
!ls

c_compiled_ls.pkl			   Juliet_Test_Suite_v1.3_for_C_Cpp.zip
cpp_compiled_ls.pkl			   ProgramData
data_acquisition			   programs.tar.gz
decodable_submisisons_ls.pkl		   raw_data.tar
drive					   sample_data
juliet_dataset_CWE_ls.pkl		   submissions_ls.pkl
juliet_dataset_CWE_testcases_paths_ls.pkl  submissions_tasks_ls.pkl
juliet_dataset_ls.pkl


##2.2 Clean New Dataset:

*   Remove Uncompilable Source Code (platform: colab(linux))

> *   Extract files from Juliet C/C++ test suite 

In [26]:
with zipfile.ZipFile('Juliet_Test_Suite_v1.3_for_C_Cpp.zip', 'r') as juliet_dataset:
   juliet_dataset.extractall()
   print('Done Extracting dataset')

Done Extracting dataset


In [27]:
#View of extracted Juliet_dataset directory
!ls /content/C

compile_all.bat		 py_common.py
create_per_cwe_files.py  run_analysis_example_tool.py
doc			 testcases
Makefile		 testcasesupport
manifest.xml		 update_main_cpp_and_testcases_h.py


In [76]:
#Check number of code files i.e. size of dataset
juliet_code_files_dir = '/content/C/testcases/'
test_file_counter = 0
for path, subdirs, files in os.walk(juliet_code_files_dir):
    for name in files:
      test_file_counter +=1
print("{} files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files".format(test_file_counter))

172675 files in unzipped Juliet archive. Matches our earlier metadata overview with zipFILE module of 106075 source code files


In [29]:
juliet_dataset_ls = load_data('juliet_dataset_ls.pkl')

In [ ]:
#Compilating CWE IDed compilable/makeFile-containing CWE IDed directories 
CWE_makefile_dir = []
cwd = os.getcwd()
print("Compiling ...")
for i in range(len(juliet_dataset_ls)):
  if juliet_dataset_ls[i].endswith('Makefile') and 'CWE' in juliet_dataset_ls[i]:
    CWE_makefile_dir.append(juliet_dataset_ls[i])
    redx = re.findall(r'\bCWE\w.*', juliet_dataset_ls[i])
    os.chdir(os.path.dirname(juliet_dataset_ls[i])) 
    print(redx[0][:-8]) 
    #Minus 8 for the length of the word Makefile leading to Makefile containing directories which are the only linun compilables
    with open(os.path.join(cwd, os.path.dirname(juliet_dataset_ls[i]),'compilation.log'), 'w') as lg:
      subprocess.run(["make", "stdout=lg"])
    os.chdir(cwd)

#152 individual CWE IDed dire with Makefiles
print("Compilated all {} Makefile containing CWE IDed directories".format(len(CWE_makefile_dir)))
print('\n')

###2.2.1 Explore Partially processed New Dataset:

In [32]:
print('Number of CWE IDed Makefiles and visualize in a List')
print(len(CWE_makefile_dir))

Number of CWE IDed Makefiles and visualize in a List
152


In [ ]:
print(CWE_makefile_dir)

In [34]:
with open('CWE_makefile_dir_ls.pkl', 'wb') as fp:
  pickle.dump (CWE_makefile_dir, fp)

## Summarize Preprocessed Data:

##Serialize & Store Preprocessed Data for next phase of the pipeline:

 >  *  1.   Use pycparser to extract ASTs from C code
 >  *  2.   Adapt Michael White's c_json.py code from pycparser Github repo to serialize ASTs


> *   pycparser and Json serializing script for single file C programs submitted in programming competition 

In [35]:
%%writefile c_json.py

#------------------------------------------------------------------------------
# pycparser: c_json.py
#
# by Michael White (@mypalmike)
#
# This example includes functions to serialize and deserialize an ast
# to and from json format. Serializing involves walking the ast and converting
# each node from a python Node object into a python dict. Deserializing
# involves the opposite conversion, walking the tree formed by the
# dict and converting each dict into the specific Node object it represents.
# The dict itself is serialized and deserialized using the python json module.
#
# The dict representation is a fairly direct transformation of the object
# attributes. Each node in the dict gets one metadata field referring to the
# specific node class name, _nodetype. Each local attribute (i.e. not linking
# to child nodes) has a string value or array of string values. Each child
# attribute is either another dict or an array of dicts, exactly as in the
# Node object representation. The "coord" attribute, representing the
# node's location within the source code, is serialized/deserialized from
# a Coord object into a string of the format "filename:line[:column]".
#
# Example TypeDecl node, with IdentifierType child node, represented as a dict:
#     "type": {
#         "_nodetype": "TypeDecl",
#         "coord": "c_files/funky.c:8",
#         "declname": "o",
#         "quals": [],
#         "type": {
#             "_nodetype": "IdentifierType",
#             "coord": "c_files/funky.c:8",
#             "names": [
#                 "char"
#             ]
#         }
#     }
#------------------------------------------------------------------------------
#from __future__ import print_function

import json
import sys
import re



from pycparser import parse_file, c_ast
from pycparser.plyparser import Coord


RE_CHILD_ARRAY = re.compile(r'(.*)\[(.*)\]')
RE_INTERNAL_ATTR = re.compile('__.*__')


class CJsonError(Exception):
    pass


def memodict(fn):
    """ Fast memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = fn(key)
            return ret
    return memodict().__getitem__


@memodict
def child_attrs_of(klass):
    """
    Given a Node class, get a set of child attrs.
    Memoized to avoid highly repetitive string manipulation
    """
    non_child_attrs = set(klass.attr_names)
    all_attrs = set([i for i in klass.__slots__ if not RE_INTERNAL_ATTR.match(i)])
    return all_attrs - non_child_attrs


def to_dict(node):
    """ Recursively convert an ast into dict representation. """
    klass = node.__class__

    result = {}

    # Metadata
    result['_nodetype'] = klass.__name__

    # Local node attributes
    for attr in klass.attr_names:
        result[attr] = getattr(node, attr)

    # Coord object
    if node.coord:
        result['coord'] = str(node.coord)
    else:
        result['coord'] = None

    # Child attributes
    for child_name, child in node.children():
        # Child strings are either simple (e.g. 'value') or arrays (e.g. 'block_items[1]')
        match = RE_CHILD_ARRAY.match(child_name)
        if match:
            array_name, array_index = match.groups()
            array_index = int(array_index)
            # arrays come in order, so we verify and append.
            result[array_name] = result.get(array_name, [])
            if array_index != len(result[array_name]):
                raise CJsonError('Internal ast error. Array {} out of order. '
                    'Expected index {}, got {}'.format(
                    array_name, len(result[array_name]), array_index))
            result[array_name].append(to_dict(child))
        else:
            result[child_name] = to_dict(child)

    # Any child attributes that were missing need "None" values in the json.
    for child_attr in child_attrs_of(klass):
        if child_attr not in result:
            result[child_attr] = None

    return result


def to_json(node, **kwargs):
    """ Convert ast node to json string """
    return json.dumps(to_dict(node), **kwargs)


def file_to_dict(filename):
    """ Load C file into dict representation of ast """
    ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_dict(ast)


def file_to_json(filename, **kwargs):
    """ Load C file into json string representation of ast """
     ## ast = parse_file(filename, use_cpp=True) my addition of pycparser fakehead to allow for intraction with files
    ast = parse_file(filename, use_cpp=True,
            cpp_path='gcc',
            cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return to_json(ast, **kwargs)


def _parse_coord(coord_str):
    """ Parse coord string (file:line[:column]) into Coord object. """
    if coord_str is None:
        return None

    vals = coord_str.split(':')
    vals.extend([None] * 3)
    filename, line, column = vals[:3]
    return Coord(filename, line, column)


def _convert_to_obj(value):
    """
    Convert an object in the dict representation into an object.
    Note: Mutually recursive with from_dict.
    """
    value_type = type(value)
    if value_type == dict:
        return from_dict(value)
    elif value_type == list:
        return [_convert_to_obj(item) for item in value]
    else:
        # String
        return value


def from_dict(node_dict):
    """ Recursively build an ast from dict representation """
    class_name = node_dict.pop('_nodetype')

    klass = getattr(c_ast, class_name)

    # Create a new dict containing the key-value pairs which we can pass
    # to node constructors.
    objs = {}
    for key, value in node_dict.items():
        if key == 'coord':
            objs[key] = _parse_coord(value)
        else:
            objs[key] = _convert_to_obj(value)

    # Use keyword parameters, which works thanks to beautifully consistent
    # ast Node initializers.
    return klass(**objs)


def from_json(ast_json):
    """ Build an ast from json string representation """
    return from_dict(json.loads(ast_json))


#------------------------------------------------------------------------------
if __name__ == "__main__":
    if len(sys.argv) > 1:
        # Some test code...
        # Do trip from C -> ast -> dict -> ast -> json, then print.
        ast_dict = file_to_dict(sys.argv[1])
        ast = from_dict(ast_dict)
        json_object = to_json(ast, sort_keys=True, indent=4)
        print(json_object)
        with open(sys.argv[1][:-2]+ ".json", "w") as outfile:
          outfile.write(json_object)
        ## little adjustment to save to a file names almost as input file. The minus 2 for ".c"
    else:
        print("Please provide a filename as argument")

Writing c_json.py


##i.) Explore Serializing of Main Dataset


In [36]:
!ls

C			      juliet_dataset_CWE_testcases_paths_ls.pkl
c_compiled_ls.pkl	      juliet_dataset_ls.pkl
c_json.py		      Juliet_Test_Suite_v1.3_for_C_Cpp.zip
cpp_compiled_ls.pkl	      ProgramData
CWE_makefile_dir_ls.pkl       programs.tar.gz
data_acquisition	      raw_data.tar
decodable_submisisons_ls.pkl  sample_data
drive			      submissions_ls.pkl
juliet_dataset_CWE_ls.pkl     submissions_tasks_ls.pkl


In [ ]:
#Test AST json generating script 
!python3 c_json.py /content/ProgramData/1/1076.c

In [ ]:
#Check that json ast (1076.json) was created in addtion to the stdout priprint 
!ls ./ProgramData/1/

In [39]:
#Generate the rest of the json ASTs from the Programming Competition submissions 
for i in range(len(compiled_c_code)):
    subprocess.run(["python3", "c_json.py", compiled_c_code[i]])
print('Done generating json ASTs. Verify with ls cmd')  

Done generating json ASTs. Verify with ls cmd


In [41]:
#Verify success of json generating script
programming_competition_c_ast_jsons = []
paths = Path('./ProgramData').glob('**/*.json')
for path in paths:
  programming_competition_c_ast_jsons.append(str(path))
  # str because path is an object not string
print("Generated {} jsons of C ASTs from the programming submissions".format(len(programming_competition_c_ast_jsons)))

Generated 36594 jsons of C ASTs from the programming submissions


In [ ]:
print(programming_competition_c_ast_jsons)
print('\n')

In [ ]:
#check for json ASTs in  ProgramData subdirectories e.g. ProgramData/1/
!ls ./ProgramData/1/ 

In [44]:
with open('programming_competition_c_ast_jsons_ls.pkl', 'wb') as fp:
  pickle.dump (programming_competition_c_ast_jsons, fp)

##Create archive of Preprocessed data for use in next phase of pipeline



In [46]:
%rm -rf data_acquisition/ raw_data.tar programs.tar.gz Juliet_Test_Suite_v1.3_for_C_Cpp.zip

In [47]:
!ls

C			      juliet_dataset_CWE_testcases_paths_ls.pkl
c_compiled_ls.pkl	      juliet_dataset_ls.pkl
c_json.py		      ProgramData
cpp_compiled_ls.pkl	      programming_competition_c_ast_jsons_ls.pkl
CWE_makefile_dir_ls.pkl       sample_data
decodable_submisisons_ls.pkl  submissions_ls.pkl
drive			      submissions_tasks_ls.pkl
juliet_dataset_CWE_ls.pkl


In [49]:
#create folder to store preprocessed_main_data
!mkdir preprocessed_main_data

In [50]:
#create folder to store preprocessed_new_data
!mkdir preprocessed_new_data

In [51]:
#Move preprocessed main data and pickled meta data to folder for compression and storage for use in next part of the pipeline
%cp -r ProgramData programming_competition_c_ast_jsons_ls.pkl c_compiled_ls.pkl cpp_compiled_ls.pkl decodable_submisisons_ls.pkl submissions_ls.pkl submissions_tasks_ls.pkl c_json.py preprocessed_main_data/

In [57]:
#Move preprocessed new data and pickled meta data to folder for compression and storage for use in next part of the pipeline
%cp -r C c_json.py CWE_makefile_dir_ls.pkl juliet_dataset_CWE_ls.pkl juliet_dataset_CWE_testcases_paths_ls.pkl juliet_dataset_ls.pkl preprocessed_new_data/

In [58]:
!ls

C
c_compiled_ls.pkl
c_json.py
cpp_compiled_ls.pkl
CWE_makefile_dir_ls.pkl
decodable_submisisons_ls.pkl
drive
juliet_dataset_CWE_ls.pkl
juliet_dataset_CWE_testcases_paths_ls.pkl
juliet_dataset_ls.pkl
preprocessed_main_data
preprocessed_new_data
ProgramData
programming_competition_c_ast_jsons_ls.pkl
sample_data
submissions_ls.pkl
submissions_tasks_ls.pkl


In [59]:
shutil.make_archive('interim_main_data','tar','/content/','preprocessed_main_data')

'/content/interim_main_data.tar'

In [ ]:
#Verify tarball of archived preprocessed_main_data
temp_archive = []
with tarfile.open('interim_main_data.tar', 'r') as green:
  for member in green:
    if member.isdir() and member.name.count ('/') > 0:
     # The '/' > 0 is to ensure the root directory is ignored 
       print(member.name) 
    if member.isfile():
     temp_archive.append (member.name)
print("\n Number of files in preprocessed_main_data tarball for export to googledrive for use later {0}".format(len(temp_archive)))

In [72]:
!cp interim_main_data.tar "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/interim/"

In [73]:
shutil.make_archive('interim_new_data','tar','/content/','preprocessed_new_data')

'/content/interim_new_data.tar'

In [ ]:
#Verify tarball of archived preprocessed_new_data
temp_archive = []
with tarfile.open('interim_new_data.tar', 'r') as green:
  for member in green:
    if member.isdir() and member.name.count ('/') > 0:
     # The '/' > 0 is to ensure the root directory is ignored 
       print(member.name) 
    if member.isfile():
     temp_archive.append (member.name)
print("\n Number of files in preprocessed_new_data tarball for export to googledrive for use later {0}".format(len(temp_archive)))

In [75]:
!cp interim_new_data.tar "/content/drive/My Drive/colab_root/compchem-compsci-shared-rep/interim/"